In [98]:
import pandas as pd
import yaml
from tabulate import tabulate

In [99]:
TVT_no_reducer_data = []
datasets = ['kuhar', 'motionsense', 'uci', 'wisdm', 'realworld_thigh', 'realworld_waist']
for dataset in datasets:
    with open(f'../execute_once_experiments/TVT_sb_tdom_no_reducer/scores/TVT_sb_no_reducer_{dataset}.yaml') as f:
        scores = yaml.load(f, Loader=yaml.FullLoader)
        TVT_no_reducer_data.append([dataset, scores['score']])
        print(dataset, scores['score'])

kuhar 0.7965277777777777
motionsense 0.8872881355932204
uci 0.8855072463768117
wisdm 0.7693990755007704
realworld_thigh 0.6959972394755003
realworld_waist 0.663233024691358


In [100]:
print(tabulate(pd.DataFrame(TVT_no_reducer_data).T, headers=['Dataset', 'Score'], tablefmt='latex', floatfmt=".2f"))

\begin{tabular}{rllllll}
\hline
    &                    &                    &                    &                    & Dataset            & Score             \\
\hline
  0 & kuhar              & motionsense        & uci                & wisdm              & realworld\_thigh    & realworld\_waist   \\
  1 & 0.7965277777777777 & 0.8872881355932204 & 0.8855072463768117 & 0.7693990755007704 & 0.6959972394755003 & 0.663233024691358 \\
\hline
\end{tabular}


In [101]:
datasets = ['kuhar', 'motionsense', 'uci', 'wisdm', 'realworld_thigh', 'realworld_waist']
models = ['umap', 'ae', 'tae', 'convae', 'convtae']
percentages = [2.5, 5, 25, 50, 75, 100, 200]

data = []
new_data = {
    'dataset': None,
    'model': None,
    'percentage': None,
    'score': None,
    'knn': None,
    'rf': None,
    'svm': None
}
for dataset in datasets:
    new_data['dataset'] = dataset
    for model in models:
        new_data['model'] = model
        for percentage in percentages:
            new_data['percentage'] = percentage
            filename = f'../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_{model}_{dataset}_P{percentage}.yaml'        
            with open(filename) as f:
                try:
                    scores = yaml.load(f, Loader=yaml.FullLoader)
                    new_data['score'] = scores['score']
                    new_data['knn'] = scores['KNN-5-accuracy (mean)']
                    new_data['rf'] = scores['randomforest-100-accuracy (mean)']
                    new_data['svm'] = scores['SVM-rbf-C1.0-accuracy (mean)']
                except:
                    print(f'Error with {filename}')
                    new_data['score'] = -1
                    new_data['knn'] = -1
                    new_data['rf'] = -1
                    new_data['svm'] = -1
                data.append(new_data.copy())

Error with ../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_umap_wisdm_P2.5.yaml
Error with ../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_umap_wisdm_P5.yaml
Error with ../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_umap_wisdm_P25.yaml
Error with ../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_umap_wisdm_P50.yaml
Error with ../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_umap_wisdm_P75.yaml
Error with ../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_umap_wisdm_P100.yaml
Error with ../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_umap_wisdm_P200.yaml


Error with ../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_umap_realworld_waist_P200.yaml


In [102]:
data = pd.DataFrame(data)
# data = data[data['dataset'] == 'uci']
# data.loc[data['score'].argmin()]

In [103]:
data

,dataset,model,percentage,score,knn,rf,svm
0,kuhar,umap,2.5,0.619444,0.583333,0.619444,0.534722
1,kuhar,umap,5.0,0.590278,0.576389,0.590278,0.520833
2,kuhar,umap,25.0,0.590972,0.583333,0.590972,0.520833
3,kuhar,umap,50.0,0.606250,0.590278,0.606250,0.527778
4,kuhar,umap,75.0,0.597222,0.597222,0.585417,0.527778
...,...,...,...,...,...,...,...
205,realworld_waist,convtae,25.0,0.602778,0.555556,0.602778,0.560571
206,realworld_waist,convtae,50.0,0.611574,0.559799,0.611574,0.553627
207,realworld_waist,convtae,75.0,0.582948,0.531636,0.567940,0.582948
208,realworld_waist,convtae,100.0,0.582948,0.579861,0.582948,0.532022


In [104]:
max_df = data.groupby(['dataset', 'model']).max()[['score']]
# print(tabulate(max_df, headers='keys', tablefmt='github'))
max_pivot = max_df.pivot_table(index='model', columns='dataset', values='score')
print(tabulate(max_pivot, headers='keys', tablefmt='latex', floatfmt=".2f"))
# [['dataset', 'model', 'score']].reset_index()

\begin{tabular}{lrrrrrr}
\hline
 model   &   kuhar &   motionsense &   realworld\_thigh &   realworld\_waist &   uci &   wisdm \\
\hline
 ae      &    0.72 &          0.81 &              0.68 &              0.61 &  0.76 &    0.72 \\
 convae  &    0.70 &          0.82 &              0.71 &              0.66 &  0.79 &    0.74 \\
 convtae &    0.69 &          0.85 &              0.62 &              0.62 &  0.78 &    0.79 \\
 tae     &    0.69 &          0.75 &              0.67 &              0.66 &  0.70 &    0.72 \\
 umap    &    0.62 &          0.77 &              0.64 &              0.54 &  0.67 &   -1.00 \\
\hline
\end{tabular}


In [105]:
data = pd.DataFrame(data)
# Add column with best string for each row
data['best'] = data[['knn', 'rf', 'svm']].idxmax(axis=1)
rows = []
for key, group in data.groupby(['model', 'percentage']):
    group.drop(['model', 'percentage', 'knn', 'rf', 'svm'], axis=1, inplace=True)
    # Round numbers
    
    group = group.set_index('dataset')
    group['score'] = group['score'].apply(lambda x: round(x*100, 1))
    group['VALUE'] = (group['score']).astype(str) + '%(' + group['best'].apply(str.upper) + ')'
    group = group.drop(['score', 'best'], axis=1)
    # print(group)
    group = group.T
    
    # print(group.loc['best'], '\nnnn')
    # Add key
    group['model'] = key[0]
    group['percentage'] = key[1]
    rows.append(group)
table3 = pd.concat(rows)
# Reorder columns
# print(tabulate(table3, headers='keys', tablefmt='psql'))
table3 = table3[['model', 'percentage', 'uci', 'motionsense', 'kuhar', 'wisdm', 'realworld_thigh', 'realworld_waist']]
table3 = table3.reset_index(drop=True)
# Change column names
table3 = table3.rename(columns={'uci': 'UCI', 'motionsense': 'MS', 'kuhar': 'KH', 'wisdm': 'WISDM', 'realworld_thigh': 'RW Thigh', 'realworld_waist': 'RW Waist'})
print(tabulate(table3, headers='keys', tablefmt='latex'))
# display(table3)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    


\begin{tabular}{rlrllllll}
\hline
    & model   &   percentage & UCI        & MS         & KH         & WISDM        & RW Thigh   & RW Waist     \\
\hline
  0 & ae      &          2.5 & 66.3\%(RF)  & 73.7\%(RF)  & 61.3\%(RF)  & 65.4\%(RF)    & 61.8\%(RF)  & 57.5\%(SVM)   \\
  1 & ae      &          5   & 73.0\%(SVM) & 76.1\%(KNN) & 67.4\%(SVM) & 69.5\%(RF)    & 64.3\%(RF)  & 60.9\%(RF)    \\
  2 & ae      &         25   & 71.8\%(RF)  & 80.7\%(KNN) & 63.9\%(KNN) & 72.4\%(KNN)   & 67.5\%(RF)  & 60.3\%(SVM)   \\
  3 & ae      &         50   & 75.8\%(SVM) & 80.0\%(KNN) & 71.5\%(KNN) & 70.4\%(RF)    & 66.2\%(RF)  & 60.1\%(SVM)   \\
  4 & ae      &         75   & 75.1\%(KNN) & 78.7\%(RF)  & 66.7\%(SVM) & 70.2\%(KNN)   & 68.5\%(RF)  & 61.0\%(SVM)   \\
  5 & ae      &        100   & 75.8\%(KNN) & 80.6\%(KNN) & 66.2\%(RF)  & 70.4\%(KNN)   & 67.6\%(RF)  & 61.0\%(SVM)   \\
  6 & ae      &        200   & 76.1\%(KNN) & 80.5\%(KNN) & 66.7\%(KNN) & 70.0\%(RF)    & 65.9\%(RF)  & 60.6\%(SVM)   \\
  7 &

In [106]:
data

,dataset,model,percentage,score,knn,rf,svm,best
0,kuhar,umap,2.5,0.619444,0.583333,0.619444,0.534722,rf
1,kuhar,umap,5.0,0.590278,0.576389,0.590278,0.520833,rf
2,kuhar,umap,25.0,0.590972,0.583333,0.590972,0.520833,rf
3,kuhar,umap,50.0,0.606250,0.590278,0.606250,0.527778,rf
4,kuhar,umap,75.0,0.597222,0.597222,0.585417,0.527778,knn
...,...,...,...,...,...,...,...,...
205,realworld_waist,convtae,25.0,0.602778,0.555556,0.602778,0.560571,rf
206,realworld_waist,convtae,50.0,0.611574,0.559799,0.611574,0.553627,rf
207,realworld_waist,convtae,75.0,0.582948,0.531636,0.567940,0.582948,svm
208,realworld_waist,convtae,100.0,0.582948,0.579861,0.582948,0.532022,rf


# Fdom

In [107]:
datasets = ['kuhar', 'motionsense', 'uci', 'wisdm', 'realworld_thigh', 'realworld_waist']
datasets = ['wisdm', 'realworld_thigh', 'realworld_waist']

data = []
new_data = {
    'dataset': None,
    'experiment': None,
    'knn': None,
    'rf': None,
    'svm': None,
    'score': None
}
experiments = ['TVT_sb_fdom_no_reducer', 'TVT_sb_fdom_acc_no_reducer', 'TVT_sb_fdom_gyr_no_reducer'] 
for dataset in datasets:
    for experiment in experiments:
        filename = f'../execute_once_experiments/{experiment}/scores/TVT_sb_no_reducer_{dataset}.yaml'
        with open(filename) as f:
            scores = yaml.load(f, Loader=yaml.FullLoader)
            new_data['dataset'] = dataset
            new_data['experiment'] = experiment
            new_data['score'] = scores['score']
            new_data['knn'] = scores['KNN-5-accuracy (mean)']
            new_data['rf'] = scores['randomforest-100-accuracy (mean)']
            new_data['svm'] = scores['SVM-rbf-C1.0-accuracy (mean)']
            data.append(new_data.copy())

In [108]:
print(tabulate(pd.DataFrame(data), headers='keys', tablefmt='psql'))

+----+-----------------+----------------------------+----------+----------+----------+----------+
|    | dataset         | experiment                 |      knn |       rf |      svm |    score |
|----+-----------------+----------------------------+----------+----------+----------+----------|
|  0 | wisdm           | TVT_sb_fdom_no_reducer     | 0.862866 | 0.857134 | 0.804314 | 0.862866 |
|  1 | wisdm           | TVT_sb_fdom_acc_no_reducer | 0.832666 | 0.834977 | 0.771957 | 0.834977 |
|  2 | wisdm           | TVT_sb_fdom_gyr_no_reducer | 0.82681  | 0.829615 | 0.796302 | 0.829615 |
|  3 | realworld_thigh | TVT_sb_fdom_no_reducer     | 0.637336 | 0.802692 | 0.748102 | 0.802692 |
|  4 | realworld_thigh | TVT_sb_fdom_acc_no_reducer | 0.753278 | 0.792892 | 0.730504 | 0.792892 |
|  5 | realworld_thigh | TVT_sb_fdom_gyr_no_reducer | 0.542443 | 0.708213 | 0.704624 | 0.708213 |
|  6 | realworld_waist | TVT_sb_fdom_no_reducer     | 0.736497 | 0.738503 | 0.743827 | 0.743827 |
|  7 | realworld_wai